In [2]:
import sympy as sp


状态空间模型

In [3]:
x1,x2,x3,x4 = sp.symbols('x1 x2 x3 x4')
u, mp, L, mc, g = sp.symbols('u mp L mc g')

# 定义状态空间模型
x1_dot = x3
x2_dot = -x4
x3_dot = (u + mp * sp.sin(x2) * (L * x4**2 - g * sp.cos(x2))) / (mc + mp * sp.sin(x2)**2)
x4_dot = (u * sp.cos(x2) + mp * L * x4**2 * sp.cos(x2) * sp.sin(x2) - (mc + mp) * g * sp.sin(x2)) / (L * mc + L * mp * sp.sin(x2)**2)

print(f'x1_dot: ', x1_dot)
print(f'x2_dot: ', x2_dot)
print(f'x3_dot: ', x3_dot)
print(f'x4_dot: ', x4_dot)

x1_dot:  x3
x2_dot:  -x4
x3_dot:  (mp*(L*x4**2 - g*cos(x2))*sin(x2) + u)/(mc + mp*sin(x2)**2)
x4_dot:  (L*mp*x4**2*sin(x2)*cos(x2) - g*(mc + mp)*sin(x2) + u*cos(x2))/(L*mc + L*mp*sin(x2)**2)


求偏导，在x = 0，u = 0处线性化

In [4]:
#计算关于 x2 和 x4 的偏导数

pf3x2 = sp.diff(x3_dot, x2)
pf3x4 = sp.diff(x3_dot, x4)
print(f'pf3x2: ', pf3x2)
print(f'pf3x4: ', pf3x4)

pf4x2 = sp.diff(x4_dot, x2)
pf4x4 = sp.diff(x4_dot, x4)
print(f'pf4x2: ', pf4x2)
print(f'pf4x4: ', pf4x4)

pf3x2:  -2*mp*(mp*(L*x4**2 - g*cos(x2))*sin(x2) + u)*sin(x2)*cos(x2)/(mc + mp*sin(x2)**2)**2 + (g*mp*sin(x2)**2 + mp*(L*x4**2 - g*cos(x2))*cos(x2))/(mc + mp*sin(x2)**2)
pf3x4:  2*L*mp*x4*sin(x2)/(mc + mp*sin(x2)**2)
pf4x2:  -2*L*mp*(L*mp*x4**2*sin(x2)*cos(x2) - g*(mc + mp)*sin(x2) + u*cos(x2))*sin(x2)*cos(x2)/(L*mc + L*mp*sin(x2)**2)**2 + (-L*mp*x4**2*sin(x2)**2 + L*mp*x4**2*cos(x2)**2 - g*(mc + mp)*cos(x2) - u*sin(x2))/(L*mc + L*mp*sin(x2)**2)
pf4x4:  2*L*mp*x4*sin(x2)*cos(x2)/(L*mc + L*mp*sin(x2)**2)
